* ```ResearchAgent```: Phân tích tài liệu để trích xuất thông tin như Sản phẩm/dịch vụ của công ty, đặc điểm nổi bật, case studies hoặc các thành công đạt được.
* ```StrategyAgent```: Lên kế hoạch chiến lược cho email bao gồm: Thông điệp chính, giọng văn và cách tiếp cận, các điểm cụ thể cần đưa vào, cấu trúc email.
* ```WriterAgent```: Viết nội dung email dựa trên chiến lược và nghiên cứu, tiêu đề phải thu hút, lời chào cá nhân hóa, nội dung hấp dẫn.

### 1. Import libraries

In [1]:
import os
import json
from typing import TypedDict, Annotated, Sequence, Union, List
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import JsonOutputParser
from langgraph.graph import StateGraph, START, END



In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### 2. Load documents and create vectorstore

In [3]:
def load_documents(folder_path: str):
    loader = DirectoryLoader(path=folder_path, glob="./*.txt", 
                             loader_cls=lambda path: TextLoader(file_path=path, encoding="utf-8"))
    documents = loader.load()
    return documents

def create_vector_store(documents: List[Document], db_dir_persist: str="db"):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    documents = text_splitter.split_documents(documents=documents)
    embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

    vectorstore = Chroma.from_documents(
        documents=documents, 
        embedding=embedding, 
        persist_directory=db_dir_persist
    )
    return vectorstore

In [4]:
llm = ChatOpenAI(temperature=0.7)


/tmp/ipykernel_61451/1291930362.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(temperature=0.7)


### 3. Create state graph

In [5]:
class AgentSate(TypedDict):
    language: str
    messages: List[str]
    topic: str
    recipient_type: str
    research_findings: str
    email_stragety: str
    final_email: str
    current_agent: str
    next_agent: str
    done: bool

### 4. Create agents

In [6]:
documents = load_documents(folder_path="data")
vector_store = create_vector_store(documents=documents)

state = AgentSate(
    language="Vietnamese",
    messages=[],
    topic="Devops",
    recipient_type="customer",
    research_findings="",
    email_strategy="",
    final_email="",
    current_agent="",
    next_agent="research",
    done=False
)


/tmp/ipykernel_61451/2252237509.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(model="text-embedding-ada-002")


```4.1 Research agent```

In [7]:
class ResearchAgent:

    def __init__(self, vector_store: Chroma):
        self.retriever = vector_store.as_retriever(
                search_type="similarity_score_threshold",
                search_kwargs={'score_threshold': 0.5, "k": 3}
            )
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=self.retriever,
            return_source_documents=True
        )

    def __call__(self, state: AgentSate) -> dict:
        print("Research Agent: Analyzing documents...")

        query = f"""
        Analyze the provided documents and extract key information about:
        1. Our company's services/products
        2. Key selling points
        3. Relevant case studies or success stories
        4. Any specific information related to {state['topic']}

        Format the findings in a clear, structured way. 
        """
        # Use only language {state['language']} for answer

        response = self.qa_chain.invoke({"query": query})
        state['research_findings'] = response['result']
        state['current_agent'] = "research"
        state['next_agent'] = "strategy" 
        return state
    

In [8]:
reseacher = ResearchAgent(vector_store=vector_store)
state = reseacher(state=state)
state

Research Agent: Analyzing documents...


Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


{'language': 'Vietnamese',
 'messages': [],
 'topic': 'Devops',
 'recipient_type': 'customer',
 'research_findings': '1. Dịch vụ/Sản phẩm của chúng tôi:\n- DevOps Engineering: Học phí Online 10$, Offline 20$. Nội dung học CI/CD, quản lý hệ thống tự động, Docker, Kubernetes.\n- Backend Development: Học phí Online 20$, Offline 30$. Nội dung học API, microservices, cơ sở dữ liệu, hiệu năng, bảo mật.\n- Frontend Development: Học phí Online 30$, Offline 40$. Nội dung học giao diện người dùng, framework JavaScript, responsive design.\n\n2. Điểm nổi bật:\n- Thực tiễn cao: Case study thực tế, giải quyết bài toán công việc, xây dựng portfolio.\n- Đội ngũ giảng viên chất lượng: Mentor có kinh nghiệm, làm việc tại các công ty hàng đầu, đào tạo phong phú.\n- Phương thức thanh toán linh hoạt: Thẻ tín dụng, chuyển khoản, trả góp.\n\n3. Thành công từ các case study:\n- Chú trọng vào các bài toán thực tế từ doanh nghiệp.\n- Học viên xây dựng portfolio thực tế.\n- Mentor có kinh nghiệm giúp giải quyết 

```4.2 Stragety Agent```